In [1]:
%load_ext rpy2.ipython
#https://gist.github.com/simecek/019d87c55fec3839d95bbf8489dde61d
from jupyter_pexpect_bash_console import pexpect_bash,pexpect_sudo_bash,pexpect_ssh_sudo_bash

In [3]:
%%R
R.version.string
print("email: 'Maciek Sykulski'<macieksk@gmail.com>")
length(ls())

[1] "email: 'Maciek Sykulski'<macieksk@gmail.com>"
[1] 0


In [5]:
%%R
projDir<-"/home/msykulski/msykulski/workspace/zgm/aneuploidy/"
projectPrefix<-"R_aneuploidy"

#
load.sources<-function(){
    for (f in  c("utils.R" #Contains inpar, save_new_image, etc
                 #...  
                 ) 
        ){
    source(paste(projDir,"/src/",f,sep=""))
   }
}
load.sources()

/home/msykulski/miniconda2/envs/p3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: hash-2.2.6 provided by Decision Patterns


  warnings.warn(x, RRuntimeWarning)


In [6]:
%%R
#system.time(load(paste(projDir,"/r_saved_images/",projectPrefix,".Rdata",sep="")))
lsos()
gc()

         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 297829 16.0     460000 24.6   350000 18.7
Vcells 524628  4.1    1023718  7.9   786430  6.0


In [7]:
%%R
gc()
#system.time(save_new_image())

         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 297897 16.0     592000 31.7   350000 18.7
Vcells 524562  4.1    1023718  7.9   786430  6.0


In [15]:
%%R
library(lattice)
library(latticeExtra)
library(ggplot2)
library(gplots)
#library(png)

#library(Hmisc)
library(cluster)
library(MASS)
library(reshape)

#library(lumi)
#library(lumiHumanIDMapping)
#library(lumiHumanAll.db)
#library(limma)

#library(annotate)
#library(GOstats)

#library(multtest)
library(matrixStats)
#library(extrafont)
#library(enrichvs)
#library(statmod)
#library(flexmix)

load.sources()

sessionInfo()

R version 3.3.1 (2016-06-21)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 8 (jessie)

locale:
 [1] LC_CTYPE=en_US.UTF-8          LC_NUMERIC=C                 
 [3] LC_TIME=en_US.UTF-8           LC_COLLATE=en_US.UTF-8       
 [5] LC_MONETARY=en_US.UTF-8       LC_MESSAGES=en_US.UTF-8      
 [7] LC_PAPER=en_US.UTF-8          LC_NAME=en_US.UTF-8          
 [9] LC_ADDRESS=en_US.UTF-8        LC_TELEPHONE=en_US.UTF-8     
[11] LC_MEASUREMENT=en_US.UTF-8    LC_IDENTIFICATION=en_US.UTF-8

attached base packages:
[1] parallel  tools     stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] matrixStats_0.51.0  reshape_0.8.6       latticeExtra_0.6-28
 [4] RColorBrewer_1.1-2  MASS_7.3-45         cluster_2.0.5      
 [7] gplots_3.0.1        ggplot2_2.1.0       lattice_0.20-34    
[10] fastmatch_1.1-0     hash_2.2.6         

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.7        gtools_3.5.0       bitops_1.0-6  

In [5]:
%%R 
#https://cran.r-project.org/mirrors.html
install.packages(c("reshape"), #c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest'),
                 repos="https://mirror.ibcp.fr/pub/CRAN/")
#repos='https://cran.us.r-project.org')

In [8]:
%%writefile shiny-examples/063-superzip-example/server.R
# %load shiny-examples/063-superzip-example/server.R
library(leaflet)
library(RColorBrewer)
library(scales)
library(lattice)
library(dplyr)

# Leaflet bindings are a bit slow; for now we'll just sample to compensate
set.seed(100)
zipdata <- allzips[sample.int(nrow(allzips), 10000),]
# By ordering by centile, we ensure that the (comparatively rare) SuperZIPs
# will be drawn last and thus be easier to see
zipdata <- zipdata[order(zipdata$centile),]

function(input, output, session) {

  ## Interactive Map ###########################################

  # Create the map
  output$map <- renderLeaflet({
    leaflet() %>%
      addTiles(
        urlTemplate = "//{s}.tiles.mapbox.com/v3/jcheng.map-5ebohr46/{z}/{x}/{y}.png",
        attribution = 'Maps by <a href="http://www.mapbox.com/">Mapbox</a>'
      ) %>%
      setView(lng = 24, lat = 51, zoom = 6)
  })
  
  do_onclick <-function(click) {
    
    proxy <- leafletProxy("map")
    # remove all markers and popups
    proxy %>% clearMarkers()
    proxy %>% clearShapes()
    
    # add new marker around the center
    proxy %>% addMarkers(lng=click$lng,lat = click$lat,popup='Your Starting Point')
    # add new circle
    proxy %>% addCircles(lng=click$lng, lat=click$lat,radius=(1609.344*10),color='red')
    
  }
  
  observeEvent(input$map_click, {
    
    click<-input$map_click
    output$clickInfo <- renderPrint(click)
    text <-paste("lat ",click$lat," lon ",click$lng)
    print(text)
    do_onclick(click)
    
  })
  
  #  #### what to do on shape click
  observeEvent(input$map_shape_click, {
    
    click<-input$map_shape_click
    output$clickInfo <- renderPrint(click)
    text <-paste("lat ",click$lat," lon ",click$lng)
    print(text)
    do_onclick(click)
    
  }) 

  # A reactive expression that returns the set of zips that are
  # in bounds right now
  zipsInBounds <- reactive({
    if (is.null(input$map_bounds))
      return(zipdata[FALSE,])
    bounds <- input$map_bounds
    latRng <- range(bounds$north, bounds$south)
    lngRng <- range(bounds$east, bounds$west)

    subset(zipdata,
      latitude >= latRng[1] & latitude <= latRng[2] &
        longitude >= lngRng[1] & longitude <= lngRng[2])
  })
  

  # Precalculate the breaks we'll need for the two histograms
  centileBreaks <- hist(plot = FALSE, allzips$centile, breaks = 20)$breaks

  output$histCentile <- renderPlot({
    # If no zipcodes are in view, don't plot
    if (nrow(zipsInBounds()) == 0)
      return(NULL)

    hist(zipsInBounds()$centile,
      breaks = centileBreaks,
      main = "SuperZIP score (visible zips)",
      xlab = "Percentile",
      xlim = range(allzips$centile),
      col = '#00DD00',
      border = 'white')
  })

  output$scatterCollegeIncome <- renderPlot({
    # If no zipcodes are in view, don't plot
    if (nrow(zipsInBounds()) == 0)
      return(NULL)

    print(xyplot(income ~ college, data = zipsInBounds(), xlim = range(allzips$college), ylim = range(allzips$income)))
  })

  # This observer is responsible for maintaining the circles and legend,
  # according to the variables the user has chosen to map to color and size.
  observe({
    colorBy <- input$color
    sizeBy <- input$size

    if (colorBy == "superzip") {
      # Color and palette are treated specially in the "superzip" case, because
      # the values are categorical instead of continuous.
      colorData <- ifelse(zipdata$centile >= (100 - input$threshold), "yes", "no")
      pal <- colorFactor("viridis", colorData)
    } else {
      colorData <- zipdata[[colorBy]]
      pal <- colorBin("viridis", colorData, 7, pretty = FALSE)
    }

    if (sizeBy == "superzip") {
      # Radius is treated specially in the "superzip" case.
      radius <- ifelse(zipdata$centile >= (100 - input$threshold), 30000, 3000)
    } else {
      radius <- zipdata[[sizeBy]] / max(zipdata[[sizeBy]]) * 30000
    }

    leafletProxy("map", data = zipdata) %>%
      clearShapes() %>%
      addCircles(~longitude, ~latitude, radius=radius, layerId=~zipcode,
        stroke=FALSE, fillOpacity=0.4, fillColor=pal(colorData)) %>%
      addLegend("bottomleft", pal=pal, values=colorData, title=colorBy,
        layerId="colorLegend")
  })

  # Show a popup at the given location
  showZipcodePopup <- function(zipcode, lat, lng) {
    selectedZip <- allzips[allzips$zipcode == zipcode,]
    content <- as.character(tagList(
      tags$h4("Score:", as.integer(selectedZip$centile)),
      tags$strong(HTML(sprintf("%s, %s %s",
        selectedZip$city.x, selectedZip$state.x, selectedZip$zipcode
      ))), tags$br(),
      sprintf("Median household income: %s", dollar(selectedZip$income * 1000)), tags$br(),
      sprintf("Percent of adults with BA: %s%%", as.integer(selectedZip$college)), tags$br(),
      sprintf("Adult population: %s", selectedZip$adultpop)
    ))
    leafletProxy("map") %>% addPopups(lng, lat, content, layerId = zipcode)
  }

  # When map is clicked, show a popup with city info
  observe({
    leafletProxy("map") %>% clearPopups()
    event <- input$map_shape_click
    if (is.null(event))
      return()

    isolate({
      showZipcodePopup(event$id, event$lat, event$lng)
    })
  })
  
  showLatLng <- function(lat,lng) {
    output$coord_lat <- renderText({
      paste("Latitude__:",lat)
    })
    output$coord_lng <- renderText({
      paste("Longitude:",lng)
    })
  }
  
  observe({
    proxy <- leafletProxy("map")
    proxy %>% clearPopups()
    event <- input$map_click
    isolate({
      showLatLng(event$lat,event$lng)
    })
  })


  ## Data Explorer ###########################################

  observe({
    cities <- if (is.null(input$states)) character(0) else {
      filter(cleantable, State %in% input$states) %>%
        `$`('City') %>%
        unique() %>%
        sort()
    }
    stillSelected <- isolate(input$cities[input$cities %in% cities])
    updateSelectInput(session, "cities", choices = cities,
      selected = stillSelected)
  })

  observe({
    zipcodes <- if (is.null(input$states)) character(0) else {
      cleantable %>%
        filter(State %in% input$states,
          is.null(input$cities) | City %in% input$cities) %>%
        `$`('Zipcode') %>%
        unique() %>%
        sort()
    }
    stillSelected <- isolate(input$zipcodes[input$zipcodes %in% zipcodes])
    updateSelectInput(session, "zipcodes", choices = zipcodes,
      selected = stillSelected)
  })

  observe({
    if (is.null(input$goto))
      return()
    isolate({
      map <- leafletProxy("map")
      map %>% clearPopups()
      dist <- 0.5
      zip <- input$goto$zip
      lat <- input$goto$lat
      lng <- input$goto$lng
      showZipcodePopup(zip, lat, lng)
      map %>% fitBounds(lng - dist, lat - dist, lng + dist, lat + dist)
    })
  })

  output$ziptable <- DT::renderDataTable({
    df <- cleantable %>%
      filter(
        Score >= input$minScore,
        Score <= input$maxScore,
        is.null(input$states) | State %in% input$states,
        is.null(input$cities) | City %in% input$cities,
        is.null(input$zipcodes) | Zipcode %in% input$zipcodes
      ) %>%
      mutate(Action = paste('<a class="go-map" href="" data-lat="', Lat, '" data-long="', Long, '" data-zip="', Zipcode, '"><i class="fa fa-crosshairs"></i></a>', sep=""))
    action <- DT::dataTableAjax(session, df)

    DT::datatable(df, options = list(ajax = list(url = action)), escape = FALSE)
  })
}


Overwriting shiny-examples/063-superzip-example/server.R


In [7]:
%%writefile shiny-examples/063-superzip-example/ui.R
# %load shiny-examples/063-superzip-example/ui.R
library(leaflet)

# Choices for drop-downs
vars <- c(
  "Is SuperZIP?" = "superzip",
  "Centile score" = "centile",
  "College education" = "college",
  "Median income" = "income",
  "Population" = "adultpop"
)


navbarPage("LuGi - Aviva", id="nav",

  tabPanel("Interactive map",
    div(class="outer",

      tags$head(
        # Include our custom CSS
        includeCSS("styles.css"),
        includeScript("gomap.js")
      ),

      # If not using custom CSS, set height of leafletOutput to a number instead of percent
      leafletOutput("map", width="100%", height="100%"),

      # Shiny versions prior to 0.11 should use class = "modal" instead.
      absolutePanel(id = "controls", class = "panel panel-default", fixed = TRUE,
        draggable = TRUE, top = 60, left = "auto", right = 20, bottom = "auto",
        width = 330, height = "auto",

        h2("Health myth busters"),

        selectInput("color", "Color", vars),
        selectInput("size", "Size", vars, selected = "adultpop"),
        conditionalPanel("input.color == 'superzip' || input.size == 'superzip'",
          # Only prompt for threshold when coloring or sizing by superzip
          numericInput("threshold", "SuperZIP threshold (top n percentile)", 5)
        ),
        
        textOutput("coord_lat"),
        textOutput("coord_lng"),

        plotOutput("histCentile", height = 200),
        plotOutput("scatterCollegeIncome", height = 250)
      ),

      tags$div(id="cite",
        'Data compiled for ', tags$em('Coming Apart: The State of White America, 1960–2010'), ' by Charles Murray (Crown Forum, 2012).'
      )
    )
  ),

  tabPanel("Data explorer",
    fluidRow(
      column(3,
        selectInput("states", "States", c("All states"="", structure(state.abb, names=state.name), "Washington, DC"="DC"), multiple=TRUE)
      ),
      column(3,
        conditionalPanel("input.states",
          selectInput("cities", "Cities", c("All cities"=""), multiple=TRUE)
        )
      ),
      column(3,
        conditionalPanel("input.states",
          selectInput("zipcodes", "Zipcodes", c("All zipcodes"=""), multiple=TRUE)
        )
      )
    ),
    fluidRow(
      column(1,
        numericInput("minScore", "Min score", min=0, max=100, value=0)
      ),
      column(1,
        numericInput("maxScore", "Max score", min=0, max=100, value=100)
      )
    ),
    hr(),
    DT::dataTableOutput("ziptable")
  ),

  conditionalPanel("false", icon("crosshair"))
)


Overwriting shiny-examples/063-superzip-example/ui.R


In [10]:
%%writefile shiny-examples/063-superzip-example/global.R
# %load shiny-examples/063-superzip-example/global.R
library(dplyr)

allzips <- readRDS("data/superzip.rds")
allzips$latitude <- jitter(allzips$latitude)
allzips$longitude <- jitter(allzips$longitude)
allzips$college <- allzips$college * 100
allzips$zipcode <- formatC(allzips$zipcode, width=5, format="d", flag="0")
row.names(allzips) <- allzips$zipcode

cleantable <- allzips %>%
  select(
    City = city.x,
    State = state.x,
    Zipcode = zipcode,
    Rank = rank,
    Score = centile,
    Superzip = superzip,
    Population = adultpop,
    College = college,
    Income = income,
    Lat = latitude,
    Long = longitude
  )


Overwriting shiny-examples/063-superzip-example/global.R


In [13]:
%%writefile shiny-examples/063-superzip-example/gomap.js
// %load shiny-examples/063-superzip-example/gomap.js

// When locator icon in datatable is clicked, go to that spot on the map
$(document).on("click", ".go-map", function(e) {
  e.preventDefault();
  $el = $(this);
  var lat = $el.data("lat");
  var long = $el.data("long");
  var zip = $el.data("zip");
  $($("#nav a")[0]).tab("show");
  Shiny.onInputChange("goto", {
    lat: lat,
    lng: long,
    zip: zip,
    nonce: Math.random()
  });
});


Overwriting shiny-examples/063-superzip-example/gomap.js
